# 1.3 Lifestyle analysis

The aim in the lifestyle anylisis is to resume a typical day for each user. Basically, all information related to activities and tags images has been used to study users habits to compare them.   

## 1. Prepare data to analyze

In [61]:
import pandas as pd
import numpy as np

df_act=pd.read_csv("/Users/Aneta/Desktop/Final_Project_Aneta/ntcir12.csv", header=0, sep=",")
df_act['path_id'] = df_act['image_path'].str.split('/').str.get(3)
df_act['path_id'] = df_act['path_id'].str[:33]
df_act=df_act.drop_duplicates(['path_id'], keep='last')
df_act.rename(columns={'activity': 'activity_new', 'Unnamed: 0': 'date_time'}, inplace=True)
df_act=df_act.drop('image_path', axis=1)

In [62]:
#Extract from datetime only dates to compare the user´s behaviour
df_act["date_time"]=pd.to_datetime(df_act["date_time"])
df_act['date_just'] = df_act['date_time'].dt.date

#Extract from datetime only time to analyse activities flow and daily schedule
df_act['time_good'] = df_act['date_time'].dt.time

#Extract from datetime only hours to compare the user´s behaviour 
df_act['time_hour'] = df_act["date_time"].apply(lambda x: x.hour)

#Group time of a day to a range like morning, afternoon, evening and night
df_act["time_range"]=df_act["time_hour"]
df_act["time_range"].loc[(df_act["time_range"] >= 6)& (df_act["time_range"] <= 11)]=1
df_act["time_range"].loc[(df_act["time_range"] >= 12)& (df_act["time_range"] <= 17)]=2
df_act["time_range"].loc[(df_act["time_range"] >= 18)& (df_act["time_range"] <= 22)]=3
df_act["time_range"].loc[(df_act["time_range"] >= 23)& (df_act["time_range"] <= 5)]=4
df_act["time_range"].replace([1,2,3,4,], ["morning", "afternoon", "evening", "night"], inplace=True)


#Extract a day of a week to compare the user´s behaviour 
import calendar
import time
df_act['Month'] = df_act['date_time'].dt.month.fillna(0).astype(int)
df_act['Month'] = df_act['Month'].apply(lambda x: calendar.month_abbr[x])
df_act['day_of_week_good'] = df_act['date_time'].dt.dayofweek
df_act["day_of_week_name"]=df_act['day_of_week_good']
df_act["day_of_week_name"].replace([0,1,2,3,4,5,6], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], inplace=True)

In [63]:
#The analysis will be conducted only on observations with assigned images. Other cases should be dropped
df_act=df_act.dropna(subset=['path_id'])

## 2. Lifestyle analysis

In [99]:
#List of tags from Yolo
tags=('person','chair', 'book', 'tvmonitor', 'laptop', 'bottle','cup', 'car','diningtable', 'cell phone',
             'keyboard', 'bowl', 'mouse', 'clock','toilet', 'sink', 'remote', 'suitcase', 'pottedplant','refrigerator',
             'knife', 'handbag', 'vase', 'aeroplane', 'cat','bed', 'sofa', 'backpack', 'tie', 'spoon', 'toothbrush',
             'traffic light', 'bicycle', 'train', 'bird', 'microwave', 'bench','fork', 'oven', 'motorbike', 'donut',
             'wine glass', 'pizza','apple', 'scissors', 'umbrella', 'cake', 'bus', 'truck','banana', 'parking meter',
             'sandwich', 'sports ball', 'broccoli','carrot', 'orange', 'teddy bear', 'dog', 'snowboard','skateboard', 'boat',
             'surfboard', 'frisbee', 'skis', 'hot dog','bear', 'elephant', 'toaster', 'stop sign', 'hair drier', 'kite',
             'sheep', 'zebra', 'tennis racket', 'baseball bat', 'fire hydrant','horse', 'cow', 'giraffe', 'baseball glove')



In [131]:
df_tags=df_act.drop(["age", "day_id", "image_directory", "image_id", "gender", "height", "weight", "filename",
                     "day_of_week"], axis=1)

In [133]:
df_tags1=df_tags.drop(["date_time", "date", "location", "day_of_week_name", "path_id", "time_good",
                      "time_hour", "time_range", "Month", "day_of_week_good", "date_just"], axis=1)

In [127]:
#replace values for tags
df_tags2=df_tags1.iloc[:,2:]
df_tags2=df_tags2*100
df_tags2=df_tags2.round().astype(int)
df_tags2[df_tags2 > 0] = 1

In [134]:
activity=df_tags1[["user", "activity_new"]]

In [193]:
df_tags3=pd.concat([activity,df_tags2 ], axis=1)

In [194]:
df_tags3

,user,activity_new,person,chair,book,tvmonitor,laptop,bottle,cup,car,...,kite,sheep,zebra,tennis racket,baseball bat,fire hydrant,horse,cow,giraffe,baseball glove
0,u1,mobile,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,u1,cooking,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,u1,cooking,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,u1,cooking,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,u1,cooking,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,u1,cooking,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,u1,cooking,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,u1,having drinks\meal alone,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,u1,tv,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,u1,tv,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## 3. Habits comparation

In [195]:
#List of tags from Yolo
tags=('person','chair', 'book', 'tvmonitor', 'laptop', 'bottle','cup', 'car','diningtable', 'cell phone',
             'keyboard', 'bowl', 'mouse', 'clock','toilet', 'sink', 'remote', 'suitcase', 'pottedplant','refrigerator',
             'knife', 'handbag', 'vase', 'aeroplane', 'cat','bed', 'sofa', 'backpack', 'tie', 'spoon', 'toothbrush',
             'traffic light', 'bicycle', 'train', 'bird', 'microwave', 'bench','fork', 'oven', 'motorbike', 'donut',
             'wine glass', 'pizza','apple', 'scissors', 'umbrella', 'cake', 'bus', 'truck','banana', 'parking meter',
             'sandwich', 'sports ball', 'broccoli','carrot', 'orange', 'teddy bear', 'dog', 'snowboard','skateboard', 'boat',
             'surfboard', 'frisbee', 'skis', 'hot dog','bear', 'elephant', 'toaster', 'stop sign', 'hair drier', 'kite',
             'sheep', 'zebra', 'tennis racket', 'baseball bat', 'fire hydrant','horse', 'cow', 'giraffe', 'baseball glove')

In [200]:
#spedentary
df_tags3["spedentary"]=0
df_tags3.loc[(df_tags3['chair'] == 1)|(df_tags3['tvmonitor'] == 1)|(df_tags3['laptop'] == 1)
             |(df_tags3['diningtable'] == 1)|(df_tags3['keyboard'] == 1)|(df_tags3['mouse'] == 1)
             |(df_tags3['remote'] == 1)|(df_tags3['bed'] == 1)|(df_tags3['sofa'] == 1)
             |(df_tags3['bench'] == 1)|(df_tags3['book'] == 1),
             'spedentary'] = 1

#Junk food
df_tags3["junk_food"]=0
df_tags3.loc[(df_tags3['pizza'] == 1)|(df_tags3['hot dog'] == 1)|(df_tags3['donut'] == 1)
             |(df_tags3['cake'] == 1),
             'junk_food'] = 1
#Healthy food
df_tags3["healthy_food"]=0
df_tags3.loc[(df_tags3['apple'] == 1)|(df_tags3['banana'] == 1)|(df_tags3['broccoli'] == 1)
             |(df_tags3['carrot'] == 1)|(df_tags3['orange'] == 1),
             'healthy_food'] = 1
#Sport
df_tags3["sport"]=0
df_tags3.loc[(df_tags3['sports ball'] == 1)|(df_tags3['snowboard'] == 1)|(df_tags3['skateboard'] == 1)
             |(df_tags3['boat'] == 1)|(df_tags3['surfboard'] == 1)|(df_tags3['frisbee'] == 1)
             |(df_tags3['skis'] == 1)|(df_tags3['kite'] == 1)|(df_tags3['tennis racket'] == 1)
             |(df_tags3['baseball bat'] == 1)|(df_tags3['baseball glove'] == 1)
             |(df_tags3['bicycle'] == 1),
             'sport'] = 1



In [204]:
food=df_tags3[["user","healthy_food", "junk_food"]]
food=food.groupby(food["user"]).sum()
food["Total"]=food.sum(axis=1)
food=food.div(food["Total"], axis=0)*100
food=food.round(2)
food

,healthy_food,junk_food,Total
user,,,
u1,50.27,49.73,100.0
u2,39.09,60.91,100.0
u3,39.53,60.47,100.0


In [213]:
df_tags3["free_time"]=0
df_tags3.loc[(df_tags3['activity_new'] == "resting"),
             'free_time'] = 1

active_life=df_tags3[df_tags3["free_time"]==1]

active_life=active_life[["user","sport", "spedentary"]]
active_life=active_life.groupby(active_life["user"]).sum()
active_life["Total"]=active_life.sum(axis=1)
active_life=active_life.div(active_life["Total"], axis=0)*100
active_life=active_life.round(2)
active_life

,sport,spedentary,Total
user,,,
u1,7.43,92.57,100.0
u2,17.72,82.28,100.0
u3,3.38,96.62,100.0


In [211]:
#Healthy or unhealthy social life
df_tags3["social"]=0
df_tags3.loc[(df_tags3['activity_new'] == "having drinks with somebody")|(df_tags3['activity_new'] == "socializing")
             |(df_tags3['activity_new'] == "meeting")
             |(df_tags3['activity_new'] == "having meal with somebody"),
             'social'] = 1

social=df_tags3[df_tags3["social"]==1]
health_social=social[["user","wine glass", "cup"]]
health_social=health_social.groupby(health_social["user"]).sum()
health_social["Total"]=health_social.sum(axis=1)
health_social=health_social.div(health_social["Total"], axis=0)*100
health_social=health_social.round(2)
health_social

,wine glass,cup,Total
user,,,
u1,8.79,91.21,100.0
u2,11.26,88.74,100.0
u3,17.01,82.99,100.0


In [219]:
df_tags3["social"]=0
df_tags3.loc[(df_tags3['activity_new'] == "having drinks with somebody")|(df_tags3['activity_new'] == "socializing")
             |(df_tags3['activity_new'] == "meeting")
             |(df_tags3['activity_new'] == "having meal with somebody"),
             'social'] = 1

df_tags3["job"]=0
df_tags3.loc[(df_tags3['activity_new'] == "working")|(df_tags3['activity_new'] == "attending a seminar"),
             'job'] = 1

job_social=df_tags3[["user","job", "social"]]
job_social=job_social.groupby(job_social["user"]).sum()
job_social["Total"]=job_social.sum(axis=1)
job_social=job_social.div(job_social["Total"], axis=0)*100
job_social=job_social.round(2)
job_social


,job,social,Total
user,,,
u1,50.96,49.04,100.0
u2,70.02,29.98,100.0
u3,23.87,76.13,100.0
